In [ ]:
# sudo -H anaconda/bin/pip install biopython

from Bio import Entrez
from Bio import SeqIO

In [ ]:
Entrez.email = 'greg.clinton@gmail.com'

In [ ]:
# http://www.ncbi.nlm.nih.gov/Sitemap/samplerecord.html

# How many genes does e. coli have? What are their names?
# How many species under homo?
# sra vs nucleotide, gene, ref_seq

In [3]:
q = 'human[porg] AND tp53[gene] AND biomol_rna[prop] AND srcdb_refseq[prop]' 
db = 'nuccore'

In [ ]:
q = 'human[porg] AND tp53[gene] AND biomol_rna[prop] AND srcdb_refseq[prop]' 
record = Entrez.read(Entrez.esearch(db = dv, term = q))
ids = ','.join(record['IdList'][:5])
handle = Entrez.efetch(db = 'nuccore', id = ids, rettype = 'gb', retmode = 'text')
records = SeqIO.parse(handle, 'gb')

In [ ]:
for record in records:
    print record.name

In [164]:
import requests
import json
from cStringIO import StringIO

email = 'greg.clinton@gmail.com'

def entrez(cgi, path):
    use_json = cgi != 'efetch'
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
    url += cgi + '.fcgi?'
    url += '' if path == None else ('%s&' % path) 
    url += 'tool=entrez&email=' + email
    url += '&retmode=%s' % 'json' if use_json else 'text'
    text = requests.get(url).text
    return json.loads(text)[cgi + 'result'] if use_json else text

def einfo(db = None):
    return entrez('einfo', '' if db == None else 'version=2.0&db=%s' % db)

def databases():
    return einfo()['dblist']

def fields(db):
    class Field: pass

    fields = []
    for item in einfo(db)['dbinfo']['fieldlist']:
        field = Field()
        for key, value in item.iteritems():
            setattr(field, key, value)
        fields.append(field)
    return fields

def esearch(db, term):
    res = entrez('esearch', 'db=%s&term=%s&usehistory=y' % (db, term))
    return res['webenv'], res['querykey']

def parse_gb(text):
    records = []
    stream = StringIO(text)
    
    while True:
        line = stream.readline()
        
        if line == '':
            break
            
        line = line.strip('\n')
        if line.startswith('LOCUS'):
            records.append(line)
    
    return records

def efetch(db, history, rettype = 'gb'):
    env, key = history
    path = 'db=%s&WebEnv=%s&query_key=%s&rettype=%s' % (db, env, key, rettype)
    text = entrez('efetch', path)
    return parse_gb(text) if rettype == 'gb' else text

records = efetch('nuccore', esearch('nuccore', q))

for record in records:
    print record

LOCUS       NM_001141980            6216 bp    mRNA    linear   PRI 25-AUG-2016
LOCUS       NM_001141979            6210 bp    mRNA    linear   PRI 25-AUG-2016
LOCUS       NM_005657               6236 bp    mRNA    linear   PRI 25-AUG-2016
LOCUS       NM_001276699            2331 bp    mRNA    linear   PRI 12-JUN-2016
LOCUS       NM_001276695            2651 bp    mRNA    linear   PRI 12-JUN-2016
LOCUS       NM_001126113            2651 bp    mRNA    linear   PRI 12-JUN-2016
LOCUS       NM_001126117            2331 bp    mRNA    linear   PRI 12-JUN-2016
LOCUS       XM_017022533           10470 bp    mRNA    linear   PRI 06-JUN-2016
LOCUS       XM_011521986            5967 bp    mRNA    linear   PRI 06-JUN-2016
LOCUS       XR_001751382           10533 bp    RNA     linear   PRI 06-JUN-2016
LOCUS       XR_931899               3878 bp    RNA     linear   PRI 06-JUN-2016
LOCUS       XM_011521985           10215 bp    mRNA    linear   PRI 06-JUN-2016
LOCUS       NM_000546               2591